Data Science – CIS711

Case study 2

- 

# Extract data

In [7]:
#pip install cssselect

In [8]:
import numpy as np 
import pandas as pd
import requests 
from bs4 import BeautifulSoup
import csv
from lxml import html
import re

## **Explanation of scraping tool**

In this section we will demonstrate our scraping tool and strategy by scraping data for one player. The process of scraping in the next section relies on using for loops to automate this process for many players.

The steps and tools we used for scraping are as follow:


First, We use the **request** library and the **get** function to get a response object (to grap the code of a page) and assign it to a variable 'page' and we will start with the clubs page:

In [9]:
page = requests.get('https://www.premierleague.com/clubs')

Second, we get the content of the response object using function **.content provided by Request library ('page.content')**
Then a document of this content is obtained using the **'fromstring'** function provided by **'html'** package of the **lxml** model
The result is stored in a variable 'tree'.

In [10]:
tree = html.fromstring(page.content)

Third, we used the **cssselect** method provided by **HTML** to locate and select elements and their childern from the 'tree' element we have gotten in the previous step.
for example if the elemnt we want to get is the link of a specific team from the clubs page, we use the cssselect on the class of teams which is on our case **'indexItem'**

Note: the class for each element can be obtained from the page by clikcing the right cursor of the mouse and choosing **inspect**, then we hover the mouse over the element to know its class    

In [11]:
linkLocation = tree.cssselect('.indexItem')[0].attrib['href']
linkLocation

'/clubs/1/Arsenal/overview'

Finally, this linklocation is added to the webpage link so we can get the total link of each team 

In [12]:
temp="http://www.premierleague.com/" + linkLocation
temp=temp.replace ("overview", "squad")
temp

'http://www.premierleague.com//clubs/1/Arsenal/squad'

The same approach is used to scrape players links from each team page, the difference is that the class of the playes is **'.playerOverviewCard'**

In [13]:
squadPage = requests.get(temp)
squadTree = html.fromstring(squadPage.content)
    
#Extracting players' links, they are under class 'playerOverviewCard'
playerLocation = squadTree.cssselect('.playerOverviewCard')[0].attrib['href']
print(playerLocation)
temp2="http://www.premierleague.com/" + playerLocation
temp2

/players/4985/Bernd-Leno/overview


'http://www.premierleague.com//players/4985/Bernd-Leno/overview'

Since we have the links of each player in the club we also use the same approach to get infomation on this player 
Class of Name:'div.name'
class of position: '.playerOverviewAside .info'


In [14]:
playerPage1 = requests.get(temp2)
playerTree1 = html.fromstring(playerPage1.content)
tempName = str(playerTree1.cssselect('div.name')[0].text_content())
print(tempName)
tempposition = str(playerTree1.cssselect('.playerOverviewAside .info')[1].text_content())
print(tempposition)


Bernd Leno
Goalkeeper


## **Teams' pages scraping**

In this section, we are scraping the pages for each team in Premier League. To do so , we will first access the page containing all clubs. Then , we will loop through all teams and save their links in a list so we can use them latter. In the clubs page, each team is represented by a link that takes you to the team page containing its stats and overview

In [15]:
#Take site and structure html
page = requests.get('https://www.premierleague.com/clubs')
tree = html.fromstring(page.content)

In [16]:
# extract link locations related to the teams contained in the page
linkLocation = tree.cssselect('.indexItem')

In [17]:
#Create an empty list for us to send each team's link to
teamLinks = []

# There are 20 teams, we will loop through them 
for i in range(0,20):
    
    # A variable that contains the ith team's link location
    temp = linkLocation[i].attrib['href']
    
    # To access the page of each team using the code i.e not manually, we will add the team's link to the website domain 
    temp = "http://www.premierleague.com/" + temp
    
    # When opening the team's page it will be the overview page, we will change it to squad since we are interested in players
    temp = temp.replace("overview", "squad")
    
    #Add the finished link to our teamLinks list
    teamLinks.append(temp)

## **Players' pages scraping**

In this section, we will use the links we have got for each team and we will make a list of similar links for all playeras in that team. This will help us scrape the information of each player later  

In [19]:
#Create empty lists for player links
#First list is used for links in the overview page of each player while the second is used for the stats page
#Each page contains information we will need for each player
playerLink1 = []
playerLink2 = []

#looping through teams using their links
for i in range(len(teamLinks)):
    
    #accessing each team page 
    squadPage = requests.get(teamLinks[i])
     = html.fromstring(squadPage.content)
    
    #Extracting players' links, they are under class 'playerOverviewCard'
    playerLocation = squadTree.cssselect('.playerOverviewCard')

    #loop through players links in order to add them to the website domain. This eliminates the need for accessing each players page manually
    for i in range(len(playerLocation)):
        
        # add the link to the website domain 
        playerLink1.append("http://www.premierleague.com/" + playerLocation[i].attrib['href'])
        
        


In [ ]:
for i in range(len(playerLink1)):
    temp=playerLink1[i].replace("overview", "stats")
    playerLink2.append(playerLink1[i].replace("overview", "stats"))

In [ ]:
#Create lists for each variable
Name = []
Team = []
Age = []
Apps = []
HeightCM = []
WeightKG = []
Goals= []
Assists=[]
Passes=[]
Wins=[]
Losses=[]
Yellowcards=[]
Redcards=[]
Fouls=[]
Position=[]

In [ ]:
for i in range(len(playerLink1)):

    #...download and process the two pages collected earlier...
    playerPage1 = requests.get(playerLink1[i])
    playerTree1 = html.fromstring(playerPage1.content)
    playerPage2 = requests.get(playerLink2[i])
    playerTree2 = html.fromstring(playerPage2.content)

    #...find the relevant datapoint for each player, starting with name...
    tempName = str(playerTree1.cssselect('div.name')[0].text_content())
    #print(tempName)
    
    #...and team, but if there isn't a team, return "BLANK"...
    try:
        tempTeam = str(playerTree1.cssselect('.table:nth-child(1) .long')[0].text_content())
    except IndexError:
        tempTeam = str("BLANK")
    #print(tempTeam)
    
    #...and age, but if this isn't there, leave a blank 'no number' number...
    try:  
        tempAge = playerTree1.cssselect('.pdcol2 li:nth-child(1) .info--light')[0].text_content()
        res= tempAge[tempAge.find("(")+1:tempAge.find(")")]
        #res = re.findall(r"\(\s*\+?(-?\d+)\s*\)", tempAge)
    except IndexError:
        tempAge = float('NaN')
    #print(res) 

    #...and appearances. This is a bit of a mess on the page, so tidy it first...
    try:
        tempApps = playerTree2.cssselect('.statappearances')[0].text_content()
        tempApps = int(re.search(r'\d+', tempApps).group())
    except IndexError:
        tempApps = float('NaN')
        

    #...and height. Needs tidying again...
    try:
        tempHeight = playerTree1.cssselect('.pdcol3 li:nth-child(1) .info')[0].text_content()
        tempHeight = int(re.search(r'\d+', tempHeight).group())
    except IndexError:
        tempHeight = float('NaN')

    #...and weight. Same with tidying and returning blanks if it isn't there
    try:
        tempWeight = playerTree1.cssselect('.pdcol3 li+ li .info')[0].text_content()
        tempWeight = int(re.search(r'\d+', tempWeight).group())
    except IndexError:
        tempWeight = float('NaN')
    
    try:
        tempgoals =  playerTree2.cssselect('.statgoals')[0].text_content()
        tempgoals = int(re.search(r'\d+', tempgoals).group())
    except IndexError:
        tempgoals = float('NaN')
    #print(tempgoals)
    
    try:
        tempassists =  playerTree2.cssselect('.statgoal_assist')[0].text_content()
        tempassists = int(re.search(r'\d+', tempassists).group())
    except IndexError:
        tempassists = float('NaN')
    #print(tempassists)
    
    try:
        tempapasses =  playerTree2.cssselect('.stattotal_pass')[0].text_content()
        res2=tempapasses.replace(",", "")
        tempapasses = int(re.search(r'\d+', res2).group())
        
    except IndexError:
        tempapasses = float('NaN')
    #print(tempapasses)
    
    
    try:
        tempwins = playerTree2.cssselect('.statwins')[0].text_content()
        tempwins = int(re.search(r'\d+', tempwins).group())
    except IndexError:
        tempwins = float('NaN')
    
    try:
        templosses = playerTree2.cssselect('.statlosses')[0].text_content()
        templosses = int(re.search(r'\d+', templosses).group())
    except IndexError:
        templosses = float('NaN')
        
    try:
        tempYellowcards = playerTree2.cssselect('.statyellow_card')[0].text_content()
        tempYellowcards = int(re.search(r'\d+', tempYellowcards).group())
    except IndexError:
        tempYellowcards = float('NaN')
        
    try:
        tempRedcards = playerTree2.cssselect('.statred_card')[0].text_content()
        tempRedcards = int(re.search(r'\d+', tempRedcards).group())
    except IndexError:
        tempRedcards = float('NaN')
        
    try:
        tempFouls = playerTree2.cssselect('.statfouls')[0].text_content()
        tempFouls = int(re.search(r'\d+', tempFouls).group())
    except IndexError:
        tempFouls = float('NaN')     
        
    try:
        tempposition = str(playerTree1.cssselect('.playerOverviewAside .info')[1].text_content())
    except IndexError:
        tempposition = str("BLANK")
    #print(tempposition)
    
    #Now that we have a player's full details - add them all to the lists
    
    Name.append(tempName)
    Team.append(tempTeam)
    Age.append(res)
    Apps.append(tempApps)
    HeightCM.append(tempHeight)
    WeightKG.append(tempWeight)
    Goals.append(tempgoals)
    Assists.append(tempassists)
    Passes.append(tempapasses)
    Wins.append(tempwins)
    Losses.append(templosses)
    Yellowcards.append(tempYellowcards)
    Redcards.append(tempRedcards)
    Fouls.append(tempFouls)
    Position.append(tempposition)

In [ ]:
#Create data frame from lists
df = pd.DataFrame(
    {'Name':Name,
     'Team':Team,
     'Position':Position,
     'Age':Age,
     'Apps':Apps,
     'HeightCM':HeightCM,
     'WeightKG':WeightKG,
    'Goals':Goals,
    'Assists':Assists,
    'Passes':Passes,
     'Wins':Wins,
      'Losses':Losses,
   'Yellowcards':Yellowcards,
    'Redcards':Redcards,
    'Fouls':Fouls})

#Show me the top 20 rows:

df.head(20)

In [ ]:
df.shape

In [ ]:
df.to_csv('Teams_data.csv' ,index=False )

# Pre-processing

In [ ]:
df = pd.read_csv('Teams_data.csv')

In [ ]:
df.info()

In [ ]:
#cheking for null values
df.isnull().sum()

In [ ]:
df=df[df['Apps']!=0]
df.reset_index(drop=True, inplace=True  )

In [ ]:
df.isnull().sum()

We have a missing data in HeightCM and WeightKG columns. So, we will fill them with the average height and weight for the players in the same age.

In [ ]:
# group the players by age
group = df[['HeightCM' ,'WeightKG' ,'Age']].groupby('Age').mean()
# fill the null values with the mean 
for i , r in df[df['HeightCM'].isnull()].iterrows():
    age = df.iloc[i,3]
    df.iloc[i,5] = group.loc[age , 'HeightCM']
    
for i , r in df[df['WeightKG'].isnull()].iterrows():
    age = df.iloc[i,3]
    df.iloc[i,6] = group.loc[age , 'WeightKG']
df.isnull().sum()

In [ ]:
#creating new columns

df['Wins_ratio'] = df['Wins'] / df['Apps']

df['Goals_ratio'] = df['Goals'] / df['Apps']
    

df['Yellowcardsd_ratio'] = df['Yellowcards'] / df['Apps']

    
df['Fouls_ratio'] = df['Fouls'] / df['Apps']

    

df['Passes_ratio'] = df['Passes'] / df['Apps']
  

In [ ]:
df.head()

In [ ]:
# for further work we will need some columns to be descritisized, in this section we add descritisized columns to the data
# descritizing of some columns
df['Ycards_ratio_dicretized']=pd.qcut(df['Yellowcardsd_ratio'],3, labels=['Low','Medium','High'])
df['Fouls_ratio_dicretized']=pd.qcut(df['Fouls_ratio'],3, labels=['Low','Medium','High'])

In [ ]:
df.head()

In [ ]:
df.describe() # statical sammary

In [ ]:
df.info()

Now let's see how many teams we have and how many players they contain

In [ ]:
df['Team'].value_counts()

We now will clean the teams names

In [ ]:
# remove '(Loan)' and ' U21' from teams name
# substitute & to and
df['Team'] = df['Team'].apply(lambda x : re.sub("\(Loan\)|U21",' ' ,x).strip())
df['Team'] = df['Team'].apply(lambda x : re.sub("&",'and' ,x))

In [ ]:
df['Team'].value_counts()

There is still one team with one player so we will drop it 

In [ ]:
df = df.drop(index = 395).reset_index()
df['Team'].value_counts()

In [ ]:
df.to_csv('processing_Teams_data.csv' ,index=False )

# Analysis

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
%matplotlib inline

In [ ]:
df = pd.read_csv('processing_Teams_data.csv')

In [ ]:
df.head()

check for normality useing Probability-plot

In [ ]:
plt.subplot(3,3,1)
#Fouls
data = df['Fouls_ratio'].values
stats.probplot(data , plot =plt)
plt.title('Fouls_ratio')
plt.xlabel(' ')
plt.xticks([])

plt.subplot(3,3,2)
#Goals
data = df['Goals_ratio'].values
stats.probplot(data , plot =plt)
plt.title('Goals_ratio')
plt.xlabel(' ')
plt.xticks([])

plt.subplot(3,3,3)
#Passes
data = df['Passes_ratio'].values
stats.probplot(data , plot =plt)
plt.title('Passes_ratio')
plt.xlabel(' ')
plt.xticks([])

plt.subplot(3,3,4)
#Wins
data = df['Wins_ratio'].values
stats.probplot(data , plot =plt)
plt.title('Wins_ratio')
plt.xlabel(' ')
plt.xticks([])


plt.subplot(3,3,5)
#Yellowcards
data = df['Yellowcardsd_ratio'].values
stats.probplot(data , plot =plt)
plt.title('Yellowcardsd_ratio')
plt.xlabel(' ')

plt.subplot(3,3,6)
#HeightCM
data = df['HeightCM'].values
stats.probplot(data , plot =plt)
plt.title('HeightCM')
plt.xlabel(' ')

plt.subplot(3,3,7)
#WeightKG
data = df['WeightKG'].values
stats.probplot(data , plot =plt)
plt.title('WeightKG')
plt.xlabel(' ')

plt.rcParams["figure.figsize"] = (15,10)
plt.show()

### check for normality useing D’Agostino–Pearson omnibus test

In [ ]:
print(stats.normaltest(df['WeightKG'].values))
print(stats.normaltest(df['HeightCM'].values))
print(stats.normaltest(df['Fouls_ratio'].values))
print(stats.normaltest(df['Goals_ratio'].values))
print(stats.normaltest(df['Passes_ratio'].values))
print(stats.normaltest(df['Wins_ratio'].values))
print(stats.normaltest(df['Yellowcardsd_ratio'].values))

From the first figur we can notice that Passes_ratio, WeightKG and HeightCM are somehow fitting the line but after the D’Agostino–Pearson omnibus test it turns out that both of HeightCM and Fouls_ratio are the data with normal distribution

---------------------------------------

In [ ]:
plt.subplot(1,2,1)
sns.distplot(df['HeightCM'], hist=True, kde=True)

plt.subplot(1,2,2)
sns.distplot(df['Fouls_ratio'], hist=True, kde=True)
plt.show()

___________________________________

In [ ]:
#plot the distribution of Height for each Positon
sns.catplot(x="Position", y="HeightCM", kind="box", data=df)
plt.show()

We can see that the mean of heights for defender, midfielder and forward are close to each other. In other hand for players in goalkeeper position the average is higher than hights for other positions.

______________________

In [ ]:
sns.relplot(x="Apps", y="Fouls",hue='Position' ,data=df)
plt.show()

There seems to be a positive correlatoin between the number of apps and the number of fouls commited by players in each position. The only exception for this relation is for the apps of goal keepers and their commited fouls. It is obvious that the number of fouls committed by goal keeper is low regardless of the number of games they have played. This could be attributed to the fact that goal keepers usually have less contact with other players.

________________

In [ ]:
#plt.subplot(1,2,1)
sns.catplot(x="Fouls_ratio_dicretized", y="Passes_ratio", kind="box", data=df)
#plt.subplot(1,2,2)
sns.catplot(x="Ycards_ratio_dicretized", y="Passes_ratio", kind="box", data=df)
plt.show()


____________

Now we will create new table grouped by teams contain the total of red cards and the total of yellow cards

In [ ]:
Team = df[[ 'Redcards','Yellowcards' ,'Team']].groupby('Team').agg(sum)
Team.head()

In [ ]:
x = np.arange(len(Team))
width = 0.3
plt.barh(x , Team['Yellowcards'],width, color = '#f7ea00' )
plt.barh(x-0.3 ,Team['Redcards'],width , color='#ce1212')

plt.yticks(x, Team.index)
plt.legend(["Yellow cards", "Red cards"])
plt.xlabel('Total numbers',fontsize=16)
plt.ylabel('Teams',fontsize=16)
plt.show()

# Type of study

Our case study is :
- **Observational study**; because we collected information, but did not interact with the study population (players) and without trying to affect them.

- **Retrospective study**; because we took our data from previous matches since 2015 up to this season of Premier league.

- **longitudinal investigations**; we have collected information(Apps, Goals, Fouls, wins,passes,..etc) over a period of time for 6 seasons(2015-2021), multiple times from each player.

- **Case-Control**; in our study we evaluated the teams based on certain criteria. The top four in the Premier League qualify for the Champions League group stages - plus the winners of the Champions League and Europa League, there can be a maximum of five English teams in the Champions League.


# Hypothesis Testing

In Premier league when a club wants to loan a new goalkeeper they have to put into consideration different things, depending on goalkeepers posture their heights have to be higher from others in the same team.

For each 20 team in Premier League there are 4 positions : **goalkeeper, defender, midfielder and forward**.The average amount of players' height is different from position to another as shown in the figure below. The mean for goalkeepers is 190.666667. Is this difference significant? Test at the 5% level of significance.
 

In [ ]:
#plot the distribution of Height for each Position
sns.catplot(x="Position", y="HeightCM", kind="box", data=df)
plt.show()

$H_0: \mu_1 = \mu_2  $ 

$H_1: \mu_1 \neq \mu_2 $

In [ ]:
df.describe()

In [ ]:
df.loc[df.Position == "Goalkeeper"].describe()

In [ ]:
data1 = df.loc[df.Position == "Goalkeeper"]["HeightCM"].values
data2 = df.loc[df.Position != "Goalkeeper"]["HeightCM"].values

ttest, pVal = stats.ttest_ind(data1,data2)

if pVal < 0.05:
    print("Mean of Goalkeepers' height is significantly different from the mean of other positions' height")
    print(' p value ={0} '.format(pVal))

#### So the p-value was less than alpha which means we have to reject the null hypothesis..